In [1]:
# Import libraries
import psycopg2
import getpass
import numpy as np
import os.path

In [2]:
# Create a database connection
user = 'postgres'
host = 'localhost'
port = '5433'
dbname = 'mimic'
schema = 'mimiciii'

In [3]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, port=port, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))
con.commit()
print("Connected!")

Password: ··········


Connected!


In [4]:
# load admission_ids
_adm = np.load('res/admission_ids.npy', allow_pickle=True).tolist()
admission_ids = _adm['admission_ids']
print("Loaded!")

Loaded!


In [5]:
# Item IDs from INPUTEVENTS
# CareVue Item IDs are in the range (30000, 49999)
# MetaVision Item IDs are all >=200000
query = \
'''
WITH input_itemids AS (
        SELECT DISTINCT itemid
        FROM inputevents_mv
        WHERE itemid >= 200000
        UNION
        SELECT DISTINCT itemid
        FROM inputevents_cv
        WHERE itemid >= 30000 AND itemid <= 49999
    )
SELECT DISTINCT itemid
FROM input_itemids
'''
cur = con.cursor()
cur.execute(query)
data = cur.fetchall()

input_itemid = []
for row in data:
    input_itemid.append(row[0])

print("There are {} item IDs in the INPUTEVENTS tables.".format(len(input_itemid)))

There are 3216 item IDs in the INPUTEVENTS tables.


In [6]:
# Item IDs from OUTPUTEVENTS
query = \
'''
SELECT DISTINCT itemid
FROM outputevents
'''
cur = con.cursor()
cur.execute(query)
data = cur.fetchall()

output_itemid = []
for row in data:
    output_itemid.append(row[0])

print("There are {} item IDs in the OUTPUTEVENTS table.".format(len(output_itemid)))

There are 1155 item IDs in the OUTPUTEVENTS table.


In [7]:
# Item IDs from CHARTEVENTS
query = \
'''
SELECT DISTINCT itemid
FROM chartevents
'''
cur = con.cursor()
cur.execute(query)
data = cur.fetchall()

chart_itemid = []
for row in data:
    chart_itemid.append(row[0])

print("There are {} item IDs in the CHARTEVENTS table.".format(len(chart_itemid)))

There are 6463 item IDs in the CHARTEVENTS table.


In [8]:
# Item IDs from LABEVENTS
query = \
'''
SELECT DISTINCT itemid
FROM labevents
'''
cur = con.cursor()
cur.execute(query)
data = cur.fetchall()

lab_itemid = []
for row in data:
    lab_itemid.append(row[0])

print("There are {} item IDs in the LABEVENTS table.".format(len(lab_itemid)))

There are 726 item IDs in the LABEVENTS table.


In [9]:
# Item IDs from MICROBIOLOGYEVENTS
query = \
'''
SELECT DISTINCT (spec_itemid,org_itemid,ab_itemid), spec_itemid, org_itemid, ab_itemid
FROM microbiologyevents
'''
cur = con.cursor()
cur.execute(query)
data = cur.fetchall()

microbio_itemid = []
for row in data:
    microbio_itemid.append(row[1:])

print("There are {} item IDs in the MICROBIOLOGYEVENTS table.".format(len(microbio_itemid)))

There are 9154 item IDs in the MICROBIOLOGYEVENTS table.


In [10]:
# Item IDs from PRESCRIPTIONS
query = \
'''
SELECT DISTINCT formulary_drug_cd
FROM prescriptions
'''
cur = con.cursor()
cur.execute(query)
data = cur.fetchall()

prescript_itemid = []
for row in data:
    prescript_itemid.append(row[0])

print("There are {} item IDs in the PRESCRIPTIONS table.".format(len(prescript_itemid)))

There are 3268 item IDs in the PRESCRIPTIONS table.


In [11]:
# Save the Item IDs
database = {'input':input_itemid,
            'output':output_itemid,
            'chart':chart_itemid,
            'lab':lab_itemid,
            'microbio':microbio_itemid,
            'prescript':prescript_itemid}

np.save('res/item_ids.npy',database);
print('Saved!')

Saved!
